# Traffic Congestion Forecast

In [15]:
%reset

Once deleted, variables cannot be recovered. Proceed (y/[n])? y


In [16]:
import numpy as np
import pandas as pd
import sys,os,math
import time
from datetime import datetime
from sklearn import preprocessing
from keras.utils import to_categorical

In [17]:
filename = '../traffic_dataset/train_set/0707/0707_seg_1.txt'

Data = pd.read_csv(filename, header = None, dtype = np.unicode_)
Data = Data.drop([11], axis = 1)
Data.columns = ['date','time','direction','type','linkID','length','travelTime','volumn','speed','occupancy','congestionLevel']
Data = Data.drop(columns = ['date'])
Data = Data.dropna()
Data = Data[:1000]

In [18]:
len(Data)

1000

### Vectorize the parameters

In [19]:
Data.length = np.float64(Data.length)
Data.travelTime = np.float64(Data.travelTime)
Data.volumn = np.float64(Data.volumn)
Data.speed = np.float64(Data.speed)
Data.occupancy = np.float64(Data.occupancy)

Data = Data[Data['length'] >= 0]
Data = Data[Data['travelTime'] >= 0]
Data = Data[Data['volumn'] >= 0]
Data = Data[Data['speed'] >= 0]

# Data.length = preprocessing.scale(Data.length.values.reshape(-1,1))
# Data.travelTime = preprocessing.scale(Data.travelTime.values.reshape(-1,1))
# Data.volumn = preprocessing.scale(Data.volumn.values.reshape(-1,1))
# Data.speed =  preprocessing.scale(Data.speed.values.reshape(-1,1))

In [20]:
# Data.occupancy.value_counts()
Data = Data.reset_index()
occpcy_mode = np.float64(Data[Data['occupancy'] >= 0 ].occupancy.mode())
for j in range(len(Data)):
    if Data.loc[j,'occupancy'] < 0.:
        Data.loc[j, 'occupancy'] = occpcy_mode
Data.occupancy =  preprocessing.scale(Data.occupancy.values.reshape(-1,1))
Data = Data.drop(columns = ['index'])

In [21]:
for i in range(len(Data)):
    Time = Data.loc[i,'time']
    time_strp = datetime.strptime(Time, '%H%M')
    Data.loc[i,'time'] = time_strp.hour + time_strp.minute/60
  
    sys.stdout.write('   \r')
    sys.stdout.flush()
    sys.stdout.write('{}%\r'.format(i))
    sys.stdout.flush()
    time.sleep(0.01)
    
Data.time = preprocessing.normalize(Data.time.values.reshape(-1,1), norm = 'max')

In [22]:
ID = list(Data.linkID.unique())
Dir = list(Data.direction.unique())
typ = list(Data.type.unique())
conLevel = list(Data.congestionLevel.unique())

# ID_dict = dict(zip(ID, np.arange(len(ID))))
# Dir_dict = dict(zip(Dir, np.arange(len(Dir))))
# typ_dict = dict(zip(typ, np.arange(len(typ))))
# conLevel_dict = dict(zip(conLevel, np.arange(len(conLevel))))

Data['direction'] = Data['direction'].map({'EAST_BOUND':0, 'WEST_BOUND':1, 
                                               'SOUTH_BOUND':2, 'NORTH_BOUND':3, 'UNKNOWN_DIRECTION_TYPE':4}).astype(int)
Data['type'] = Data['type'].map({'FREEWAY': 0,'RAMP': 1,'ARTERIAL': 2,'LOCAL_ROAD':3, 
                                     'FREEWAY_REVERSIBLE': 4,'FREEWAY_EXPRESS': 5}).astype(int)
Data['congestionLevel'] = Data['congestionLevel'].map({'NON_CONGESTION': 0,'LIGHT_CONGESTION': 1,'MEDIUM_CONGESTION': 2,
                                                         'HEAVY_CONGESTION': 3,'UNKNOWN_CONGESTION_LEVEL': 4 }).astype(int)                                     
Data['linkID'] = Data['linkID'].map(dict(zip(ID, np.arange(len(ID))))).astype(int)


#     Data['direct'] = Data['direct'].map(dict(zip(Dir, np.arange(len(Dir))))).astype(int)
#     Data['type'] = Data['type'].map(dict(zip(typ, np.arange(len(typ))))).astype(int)
#     Data['congestionLevel'] = Data['congestionLevel'].map(dict(zip(conLevel, np.arange(len(conLevel))))).astype(int)
#     Data['linkID'] = Data['linkID'].map(ID_dict).astype(int)
#     Data['direct'] = Data['direct'].map(Dir_dict).astype(int)
#     Data['type'] = Data['type'].map(typ_dict).astype(int)
#     Data['congestionLevel'] = Data['congestionLevel'].map(conLevel_dict).astype(int)
Data = Data.drop(Data[Data['congestionLevel'] == 4].index)

In [23]:
dirt_lst = list(Data.direction)
typ_lst = list(Data.type)
lkID_lst = list(Data.linkID)
conLevel_lst = list(Data.congestionLevel)

# test_data = [1, 3, 2, 0, 3, 2, 2, 1, 0, 1]
def get_onehot(my_list):
    
    data = np.array(my_list)
    encoded = to_categorical(data)
    return encoded

# dirt_matrx = get_onehot(dirt_lst)
# typ_matrx = get_onehot(typ_lst)
# lkID_matrx = get_onehot(lkID_lst)
conLevel_matrx = get_onehot(conLevel_lst)

In [10]:
conLevel_matrx

array([[1., 0., 0., 0.],
       [1., 0., 0., 0.],
       [1., 0., 0., 0.],
       ...,
       [1., 0., 0., 0.],
       [1., 0., 0., 0.],
       [1., 0., 0., 0.]], dtype=float32)

In [25]:
Data.head()

,time,direction,type,linkID,length,travelTime,volumn,speed,occupancy,congestionLevel
0,0.0,3,0,0,1268.0,40.0,218.0,31.292915,-0.386292,0
1,0.0,2,0,1,476.0,17.0,150.0,26.822500,-1.260048,0
2,0.0,1,0,2,4124.0,150.0,0.0,27.000000,0.036673,0
3,0.0,0,0,3,9718.0,397.0,0.0,24.000000,0.036673,1
4,0.0,1,0,4,830.0,30.0,863.0,26.822500,1.311133,0


In [26]:
Data.time.head()

0    0.0
1    0.0
2    0.0
3    0.0
4    0.0
Name: time, dtype: float64